In [1260]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime

In [1261]:
ex_rates = pd.read_csv("ex_rates.csv")
payments = pd.read_csv("payments.csv")
providers = pd.read_csv("providers.csv")

In [1262]:
ex_rates.head(5)

,rate,destination
0,0.586865,AZN
1,1.041829,EUR
2,0.128461,HKD
3,0.000712,KRW
4,0.650000,AUD


In [1263]:
payments['eventTimeRes'] = pd.to_datetime(payments['eventTimeRes'])
payments.head(5)

,eventTimeRes,amount,cur,payment,cardToken
0,2024-11-24 21:00:00,700.0,RUB,c77be33e8db2f5352ebbc0a1b3f10e3e,336d5ebc5436534e61d16e63ddfca327
1,2024-11-24 21:00:01,180.0,BRL,0ab79b6d41d6e887e84ec4a85ea33733,336d5ebc5436534e61d16e63ddfca327
2,2024-11-24 21:00:01,945.9,RUB,c9a03cbbe592f930d52416b049bc1d1a,336d5ebc5436534e61d16e63ddfca327
3,2024-11-24 21:00:01,35.0,BRL,90a88e0d5b6bfff12909878f265cc2f4,336d5ebc5436534e61d16e63ddfca327
4,2024-11-24 21:00:01,90.0,BRL,f3adde51de36a39508d11d4bf4a98edc,336d5ebc5436534e61d16e63ddfca327


In [1264]:
providers.drop(columns=['LIMIT_BY_CARD'], inplace=True)
providers['TIME'] = pd.to_datetime(providers['TIME'])
providers.head(5)

,TIME,ID,CONVERSION,AVG_TIME,MIN_SUM,MAX_SUM,LIMIT_MIN,LIMIT_MAX,COMMISSION,CURRENCY
0,2024-11-24 21:00:00,0,0.50,18.0,700.0,5500.0,61000.0,12900000.0,0.035,UZS
1,2024-11-24 21:00:00,26,0.65,26.0,700.0,6000.0,31000.0,18600000.0,0.045,MYR
2,2024-11-24 21:00:00,28,0.65,22.0,200.0,9000.0,61000.0,20900000.0,0.045,MYR
3,2024-11-24 21:00:00,29,0.65,14.0,100.0,9000.0,81000.0,14700000.0,0.030,USD
4,2024-11-24 21:00:00,30,0.50,8.0,400.0,7000.0,71000.0,31600000.0,0.050,USD


TIME - дата и время, с которого состояние провайдера становится актуальным для платежей

ID - идентификатор провайдера

CONVERSION - идентификатор провайдера (отношение успешных к неуспешным платежам)

AVG_TIME - среднее время выполнения платежа на провайдере

MIN_SUM - минимальная сумма платежа (минимальный чек покупки)

MAX_SUM - максимальная сумма платежа (максимальный чек покупки)

LIMIT_MAX - максимальная общая сумма на провайдере за сутки (при превышении значения перестают проходить платежи)

LIMIT_MIN - минимальная общая сумма на провайдере за сутки (при недостижении значения получаем штрафы за невыработку объема)

LIMIT_BY_CARD - лимит по сумме платежей в разрезе одной карты плательщика

COMMISSION - комиссия, взимаемая провайдером

CURRENCY - валюта провайдера

Понять, какая разумная CONVERSION и AVG_TIME для провайдера. Это один из самых важных критериев: как же мы распределеим те или иные параметры (они зависят друг от друга). Опираться необходимо на цель заработать денег.

Лимит - важный параметр. Штраф - 1% от минимального лимита, если лимит не достигнут



In [1265]:
providers_normalized = providers.copy()
numeric_columns = ['CONVERSION', 'AVG_TIME', 'COMMISSION']

for column in numeric_columns:
    min_val = providers_normalized[column].min()
    max_val = providers_normalized[column].max()
    providers_normalized[column] = (providers_normalized[column] - min_val) / (max_val - min_val)

In [1266]:
providers['DATE'] = providers['TIME'].dt.date
limits = providers.groupby(['ID', 'DATE', 'CURRENCY'])[['LIMIT_MIN', 'LIMIT_MAX']].first()
providers[['LIMIT_MIN', 'LIMIT_MAX']] = providers.apply(
    lambda row: limits.loc[(row['ID'], row['DATE'], row['CURRENCY'])], axis=1)
providers['PENALTY'] = providers['LIMIT_MIN'] * 0.01
providers.drop(columns=['DATE'], inplace=True)

In [1267]:
limits['LIMIT_CURR'] = 0
limits

LIMIT_MIN   LIMIT_MAX  LIMIT_CURR
ID DATE       CURRENCY                                   
0  2024-11-24 UZS         61000.0  12900000.0           0
   2024-11-25 RUB         91000.0  24700000.0           0
              UZS         91000.0  15600000.0           0
   2024-11-26 RUB         11000.0  37400000.0           0
1  2024-11-24 UZS         21000.0  18000000.0           0
...                           ...         ...         ...
45 2024-11-25 PEN         81000.0  33300000.0           0
46 2024-11-24 PEN         31000.0   8300000.0           0
   2024-11-25 PEN         61000.0  36200000.0           0
47 2024-11-24 PEN          1000.0  10100000.0           0
   2024-11-25 PEN         31000.0  12500000.0           0

[163 rows x 3 columns]

In [1268]:
providers.head(5)

,TIME,ID,CONVERSION,AVG_TIME,MIN_SUM,MAX_SUM,LIMIT_MIN,LIMIT_MAX,COMMISSION,CURRENCY,PENALTY
0,2024-11-24 21:00:00,0,0.50,18.0,700.0,5500.0,61000.0,12900000.0,0.035,UZS,610.0
1,2024-11-24 21:00:00,26,0.65,26.0,700.0,6000.0,31000.0,18600000.0,0.045,MYR,310.0
2,2024-11-24 21:00:00,28,0.65,22.0,200.0,9000.0,61000.0,20900000.0,0.045,MYR,610.0
3,2024-11-24 21:00:00,29,0.65,14.0,100.0,9000.0,81000.0,14700000.0,0.030,USD,810.0
4,2024-11-24 21:00:00,30,0.50,8.0,400.0,7000.0,71000.0,31600000.0,0.050,USD,710.0


In [1269]:
providers_normalized = providers.copy()
numeric_columns = ['CONVERSION', 'AVG_TIME', 'COMMISSION', 'PENALTY']

for column in numeric_columns:
    min_val = providers_normalized[column].min()
    max_val = providers_normalized[column].max()
    providers_normalized[column] = (providers_normalized[column] - min_val) / (max_val - min_val)

In [1270]:
providers_normalized.head(5)

,TIME,ID,CONVERSION,AVG_TIME,MIN_SUM,MAX_SUM,LIMIT_MIN,LIMIT_MAX,COMMISSION,CURRENCY,PENALTY
0,2024-11-24 21:00:00,0,0.0,0.5,700.0,5500.0,61000.0,12900000.0,0.571429,UZS,0.666667
1,2024-11-24 21:00:00,26,0.6,0.9,700.0,6000.0,31000.0,18600000.0,0.857143,MYR,0.333333
2,2024-11-24 21:00:00,28,0.6,0.7,200.0,9000.0,61000.0,20900000.0,0.857143,MYR,0.666667
3,2024-11-24 21:00:00,29,0.6,0.3,100.0,9000.0,81000.0,14700000.0,0.428571,USD,0.888889
4,2024-11-24 21:00:00,30,0.0,0.0,400.0,7000.0,71000.0,31600000.0,1.000000,USD,0.777778


In [1271]:
def provider_info(id_provider, payment, normalized=False):
    """
    Получает актуальную информацию о провайдере на момент платежа
    
    Args:
        id_provider (int): ID провайдера
        payment (pd.Series): Данные о платеже
        normalized (bool): Флаг, указывающий, нужно ли нормализовать данные
    Returns:
        pd.Series: Информация о провайдере
    """
    time = payment['eventTimeRes']
    curr = payment['cur']
    if normalized:
        providers_data = providers_normalized
    else:
        providers_data = providers
    providers_data = providers_data.loc[(providers_data['ID'] == id_provider) & (providers_data['CURRENCY'] == curr)].sort_values(by='TIME')
    providers_data = providers_data.loc[providers_data['TIME'] <= time]
    if not providers_data.empty:
        provider_info = providers_data.iloc[-1]
    else:
        provider_info = None
    return provider_info

In [1272]:
def metrics(payment, list_providers):
    """
    Рассчитывает метрики для платежа:
    conversion_metric - полная вероятность успешного платежа
    avg_time_metric - среднее время выполнения платежа при условии успешного платежа
    commission_metric - комиссия при условии успешного платежа

    Args:
        payment (pd.Series): Данные о платеже
        list_providers (list): Список ID провайдеров
    Returns:
        tuple: Метрики для платежа
    """
    conversions = [provider_info(list_providers[i], payment)['CONVERSION']
                    for i in range(len(list_providers))]
    avg_times = [provider_info(list_providers[i], payment)['AVG_TIME']
                    for i in range(len(list_providers))]
    commissions = [provider_info(list_providers[i], payment)['COMMISSION']
                    for i in range(len(list_providers))]
    
    conversion_metric = conversions[0]
    avg_time_metric = avg_times[0] * conversions[0]
    commission_metric = commissions[0] * conversions[0]

    probability = 1
    for i in range(1, len(list_providers)):
        probability *= conversions[i-1]
        conversion_metric += probability * conversions[i]
        avg_time_metric += probability * conversions[i] * avg_times[i]
        commission_metric += probability * conversions[i] * commissions[i]
        
    return conversion_metric, avg_time_metric, commission_metric

In [1273]:
def calculate_provider_score(provider_data, q1, q2, q3):
    conversion_score = provider_data['CONVERSION'] * q1
    avg_time_score = provider_data['AVG_TIME'] * q2
    commission_score = provider_data['COMMISSION'] * q3
    return conversion_score + avg_time_score + commission_score


In [1284]:
def choose_best_providers(providers_indices, payment, q1, q2, q3, n_providers=3):
    """
    Выбирает лучшие провайдеры для платежа
    Args:
        providers_indices (list): Список ID провайдеров
        payment (pd.Series): Данные о платеже
        q1 (float): Вес вероятности успешного платежа
        q2 (float): Вес среднего времени выполнения платежа
        q3 (float): Вес комиссии
        n_providers (int): Количество провайдеров для выбора
    Returns:
        list: Список ID лучших провайдеров
    """
    providers_scores = []
    for provider_id in providers_indices:
        provider_data = provider_info(provider_id, payment, normalized=True)
        if provider_data is None:
            continue
        limit_idx = (provider_id, provider_data['TIME'].date(), provider_data['CURRENCY'])
        if payment['cur'] != provider_data['CURRENCY']:
            continue
        if not(provider_data['MIN_SUM'] <= payment['amount'] <= provider_data['MAX_SUM']):
            continue
        if limits.at[limit_idx, 'LIMIT_CURR'] + payment['amount'] > limits.at[limit_idx, 'LIMIT_MAX']:
            continue
        provider_score = calculate_provider_score(provider_data, q1, q2, q3)
        providers_scores.append((provider_id, provider_score))
    providers_scores.sort(key=lambda x: x[1], reverse=True)
    best_providers = [x[0] for x in providers_scores[:n_providers]]
    return best_providers


In [1285]:
providers_indices = sorted(providers['ID'].unique())

In [1288]:
choose_best_providers(providers_indices, payments.iloc[0], 0.5, 0.5, 0.5)

[4, 5, 2]